In [1]:
from utils.create_features_utils import *
import pandas as pd

In [2]:
df = pd.DataFrame()

In [3]:
for i in range(2011, 2021):
    if i <=2012:
        link = 'data/' + str(i) + '.xls'
    else:
        link = 'data/' + str(i) + '.xlsx'
    df_temp = pd.read_excel(link)
    df = df.append(df_temp, sort=False)

In [4]:
df = df.reset_index()

In [5]:
df = df[df.Date.notnull()]

In [6]:
df['Date'] = df.apply(lambda row: datetime.strptime(str(row['Date']), "%Y-%m-%d %H:%M:%S").strftime("%Y/%m/%d"), axis=1)

In [7]:
df.reset_index(inplace=True)

In [8]:
del (df['index'])

In [9]:
df = df[df.Comment == 'Completed']

In [10]:
df.loc[:,'best_of_5'] = (df['Best of'] ==5).astype(int)
df['W3'] = pd.to_numeric(df['W3'], errors='coerce')
df['L3'] = pd.to_numeric(df['L3'], errors='coerce')

In [11]:
df = df.fillna(0)

In [12]:
print(df.dtypes)

level_0         int64
ATP             int64
Location       object
Tournament     object
Date           object
Series         object
Court          object
Surface        object
Round          object
Best of         int64
Winner         object
Loser          object
WRank         float64
LRank         float64
WPts          float64
LPts          float64
W1            float64
L1            float64
W2            float64
L2            float64
W3            float64
L3            float64
W4            float64
L4            float64
W5            float64
L5            float64
Wsets         float64
Lsets         float64
Comment        object
B365W         float64
B365L         float64
EXW            object
EXL           float64
LBW           float64
LBL           float64
PSW           float64
PSL           float64
SJW           float64
SJL           float64
MaxW          float64
MaxL          float64
AvgW          float64
AvgL          float64
best_of_5       int32
dtype: object


In [13]:
df_combined = df[['Tournament',  'Date', 'Surface', 'Round']].copy()

In [14]:
df_combined.loc[:,'player_0'] = df.apply(lambda row: get_player_1_name(row['Winner'], row['WRank'], row['Loser'], row['LRank']), axis=1)
df_combined.loc[:,'player_0_rank'] = df.apply(lambda row: get_player_1_rank(row['WRank'], row['LRank']), axis=1)
df_combined.loc[:,'player_0_odd'] = df.apply(lambda row: get_player_1_odd(row['B365W'], row['WRank'], row['B365L'], row['LRank']), axis=1)

In [15]:
df_combined.loc[:,'player_1'] = df.apply(lambda row: get_player_2_name(row['Winner'], row['WRank'], row['Loser'], row['LRank']), axis=1)
df_combined.loc[:,'player_1_rank'] = df.apply(lambda row: get_player_2_rank(row['WRank'], row['LRank']), axis=1)
df_combined.loc[:,'player_1_odd'] = df.apply(lambda row: get_player_2_odd(row['B365W'], row['WRank'], row['B365L'], row['LRank']), axis=1)

In [16]:
df_combined.loc[:,'outcome'] = df.apply(lambda row: outcome(row['WRank'], row['LRank']), axis=1)

In [17]:
df_combined = df_combined[df_combined.Tournament == 'Wimbledon']

In [18]:
df_combined = df_combined[df_combined.Date > '2010/01/01']

In [19]:
df_combined = create_features(df_combined, df)

Creating Player Career Stats All Surface
Creating Player Career Stats on Grass/Clay/Hard
Creating Player Career Stats All Surface Last 52 Weeks
Creating Player Career Stats on Grass/Clay/Hard Last 60 Weeks
Creating Player Head to Head Career Stats All Surface
Creating Player Head to Head Career Stats On Grass
Creating Difference Variables


In [20]:
print(df_combined.columns)

Index(['Tournament', 'Date', 'Surface', 'Round', 'player_0', 'player_0_rank',
       'player_0_odd', 'player_1', 'player_1_rank', 'player_1_odd', 'outcome',
       'player_0_match_win_percent', 'player_1_match_win_percent',
       'player_0_games_win_percent', 'player_1_games_win_percent',
       'player_0_5_set_match_win_percent', 'player_1_5_set_match_win_percent',
       'player_0_close_sets_percent', 'player_1_close_sets_percent',
       'player_0_match_win_percent_grass', 'player_1_match_win_percent_grass',
       'player_0_games_win_percent_grass', 'player_1_games_win_percent_grass',
       'player_0_5_set_match_win_percent_grass',
       'player_1_5_set_match_win_percent_grass',
       'player_0_close_sets_percent_grass',
       'player_1_close_sets_percent_grass', 'player_0_match_win_percent_52',
       'player_1_match_win_percent_52', 'player_0_games_win_percent_52',
       'player_1_games_win_percent_52', 'player_0_5_set_match_win_percent_52',
       'player_1_5_set_match_win

In [21]:
df_combined.head()

,Tournament,Date,Surface,Round,player_0,player_0_rank,player_0_odd,player_1,player_1_rank,player_1_odd,...,diff_5_set_match_win_percent_52,diff_close_sets_percent_52,diff_match_win_percent_grass_60,diff_games_win_percent_grass_60,diff_5_set_match_win_percent_grass_60,diff_close_sets_percent_grass_60,diff_match_win_percent_hh,diff_games_win_percent_hh,diff_match_win_percent_grass_hh,diff_games_win_percent_grass_hh
1485,Wimbledon,2011/06/20,Grass,1st Round,Andujar P.,50.0,3.50,Sweeting R.,69.0,1.28,...,0.000000,0.023810,0.000000,-0.095238,0.0,-0.166667,0.0,0.0,0.0,0.0
1486,Wimbledon,2011/06/20,Grass,1st Round,Bellucci T.,28.0,1.66,Schuettler R.,113.0,2.10,...,0.600000,0.223536,0.166667,-0.015662,0.0,0.194444,0.0,0.0,0.0,0.0
1487,Wimbledon,2011/06/20,Grass,1st Round,Fish M.,9.0,1.10,Granollers M.,279.0,6.50,...,0.266667,0.147727,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
1488,Wimbledon,2011/06/20,Grass,1st Round,Kohlschreiber P.,39.0,1.22,Istomin D.,65.0,4.00,...,0.333333,0.206897,1.000000,0.260167,0.0,0.500000,0.0,0.0,0.0,0.0
1489,Wimbledon,2011/06/20,Grass,1st Round,Muller G.,92.0,1.66,Haas T.,895.0,2.10,...,0.500000,0.025000,0.000000,-0.400000,0.0,-0.333333,0.0,0.0,0.0,0.0
